# Building Micrograd
###### I will build micrograd,an autograd engine (automatic differentiation engine), which is the heart of PyTorch, TensorFlow, etc. and then solve the exercises here given by andrej karpathy in his youtube playlist hero to zero playlist.

<p style="font-size: 12px;"><i><u>Autograd</u> - Automatic gradient. Which implements backpropagation. Micrograd is built to understand maths of neural network at the fundumental levels. So we will not use our inputs as tensors or vectors here. In modern Deep neural network tensors or vectors are used for efficiency and to make the algorithms run faster. which we will not wory for now.</i></p>

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

<p style="font-size: 12px;"> <i> The %matplotlib inline command is a "magic command" in Jupyter Notebook. Its purpose is to ensure that Matplotlib plots are displayed directly below the code cell that generates them, rather than in a separate pop-up window. This creates an integrated and cohesive workflow where your code, data, and visualizations are all in one place within the notebook.</i></p>

In [ ]:
def f(x):
    return 3*x**2 - 4*x +5

<p style="font-size: 12px;"> <i>just a random function which takes an scaler input and give an scaler output.</i></p>

In [ ]:
f(3.0)

In [ ]:
xs= np.arange(-5, 5, 0.25 )
ys=f(xs)
plt.plot(xs, ys)

<p style="font-size: 12px;"> <i> np.arrange(start,stop,step) -> np.arange(-5,5,0.25):which means This creates an array [-5,-4.5,...,4.5], starting at -5, ending before 5, with a step of 0.5. Using floating-point numbers.</i><p>

 $\lim_{h \to 0} \frac{f(x+h) - f(x)}{h} $
<p style="font-size: 12px;"> <i>This is slope/derivative of our equation </i></p>


In [ ]:
h=0.000001
x=2/3 #at x=2/3 the slope becomes almost 0.
(f(x+h)-f(x))/h

In [ ]:
#lets get a bit complex
a=2.0
b=-3.0
c=10.0
d=a*b+c
print(d)

<p style="font-size: 12px;"> <i> Now, we will figure out the derivative of d with repect to a,b and c.

$ \frac{d}{da}(d) = b $

In [ ]:
h=0.000001
a=2.0
b=-3.0
c=10.0
d1=a*b+c
a+=h
d2=a*b+c
slope=(d2-d1)/h
print("derivation of d with respect to a: ",slope)

$ \frac{d}{db}(d) = a $

In [ ]:
h=0.000001
a=2.0
b=-3.0
c=10.0
d1=a*b+c
b+=h
d2=a*b+c
slope=(d2-d1)/h
print("derivation of d with respect to b: ",slope)

$ \frac{d}{dc}(d)=1$

In [ ]:
h=0.000001
a=2.0
b=-3.0
c=10.0
d1=a*b+c
c+=h
d2=a*b+c
slope=(d2-d1)/h
print("derivation of d with recpect to c :",slope)

<p style="font-size: 12px;"> <i>Right now, it looks like we’re just reinventing floats. But the big picture is:<br>
A neural network does tons of multiplications and additions.
Example: y = w*x + b is the formula for a neuron.<br>
Later, we’ll need to do backpropagation → computing gradients (dy/dw, dy/db).
Python’s built-in float can’t keep track of these operations, but our custom Value can.<br>
 This Value class is the starting point for:
Storing a number (data).<br>
Remembering how it was created (from addition/multiplication).
So, this is step 1 of building an autograd engine</i></p>

<p style="font-size: 12px;"><i>
self.data→ stores the actual number.<br>
self._prev:<br>
Keeps track of which Value objects were used to create this one.<br>
Example: if d = a * b, then d._prev = {a, b}.<br>
This is the “computation graph” idea → each value knows its parents.<br>
self._op:
Stores the operation that created this value (+ or *).
Helps us know what happened during forward pass, and later helps with backprop.
</i></p>

In [ ]:
class Value:
    def __init__(self,data,_children=(),_op=''):
        self.data=data
        self._prev=set(_children)
        self._op=_op

    def __repr__(self):
        return f"Value(data={self.data})"
#this repr(re presenter) function is used to get output in our desired format
# the code will run even without this function but 
# we wont understand the output    
    def __add__(self,other):
        out=Value(self.data+other.data,(self,other),'+')
        return out
    
    def __mul__(self,other):
        out=Value(self.data*other.data,(self,other),'*')
        return out
a=Value(2.0)
b=Value(-3.0)
c=Value(10.0)
d=a*b+c
d

Now we will visualize, so we will need graphviz.